In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

In [21]:
# Загрузка и предобработка датасета MNIST
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [22]:
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)

test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)

In [37]:
# Определение модели CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=20, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Linear(800, 1000)
        self.fc2 = nn.Linear(1000, 10)
    
    def forward(self, x):
        out = self.layer1(x)
        out = out.view(out.size(0), -1)  # Разворачивание тензора
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [38]:
# Инициализация модели, функции потерь и оптимизатора
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [39]:
# Обучение модели
num_epochs = 5
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'**Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}**')

**Epoch [1/5], Step [100/600], Loss: 0.1453**
**Epoch [1/5], Step [200/600], Loss: 0.2021**
**Epoch [1/5], Step [300/600], Loss: 0.0433**
**Epoch [1/5], Step [400/600], Loss: 0.0600**
**Epoch [1/5], Step [500/600], Loss: 0.0925**
**Epoch [1/5], Step [600/600], Loss: 0.1161**
**Epoch [2/5], Step [100/600], Loss: 0.1325**
**Epoch [2/5], Step [200/600], Loss: 0.1146**
**Epoch [2/5], Step [300/600], Loss: 0.0284**
**Epoch [2/5], Step [400/600], Loss: 0.0083**
**Epoch [2/5], Step [500/600], Loss: 0.1519**
**Epoch [2/5], Step [600/600], Loss: 0.0566**
**Epoch [3/5], Step [100/600], Loss: 0.0436**
**Epoch [3/5], Step [200/600], Loss: 0.1716**
**Epoch [3/5], Step [300/600], Loss: 0.0544**
**Epoch [3/5], Step [400/600], Loss: 0.1409**
**Epoch [3/5], Step [500/600], Loss: 0.2481**
**Epoch [3/5], Step [600/600], Loss: 0.0173**
**Epoch [4/5], Step [100/600], Loss: 0.0094**
**Epoch [4/5], Step [200/600], Loss: 0.0819**
**Epoch [4/5], Step [300/600], Loss: 0.0225**
**Epoch [4/5], Step [400/600], Los

In [40]:
# Оценка модели
model.eval()  # Переключение в режим оценки
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'**Accuracy of the model on the 10000 test images: {100 * correct / total}%**')

**Accuracy of the model on the 10000 test images: 98.05%**
